In [1]:
from IPython.display import clear_output
from kompy import KomootConnector, Authentication
from functions import add_komoot_url_to_collection, make_html_table, make_new_gpx_pages

In [2]:
email= "benhastings17+k@gmail.com"

print("Enter komoot password")
password = input()
clear_output(wait=False)


In [3]:
connector = KomootConnector(password=password, email=email)
authenticator = Authentication(password=password, email_address=email)

In [4]:
# add urls in reverse chronological order!!!
komoot_rides = [["https://www.komoot.com/tour/2696328851", 'test']

              ]


for k in komoot_rides[::-1]:
    url = k[0]
    name = k[1]
    add_komoot_url_to_collection(url, authenticator, connector, name=name)

In [2]:
make_html_table()
#make_new_gpx_pages()

In [7]:
import pandas as pd
route_info_file='gpxs/info.csv'

df = pd.read_csv(route_info_file, dtype={'i':str})

In [8]:
df

,i,name,distance,elevation,url
0,001,Kottenforst Trails,31,357,https://rdspt.com/inlawsgravel/gpxs/001.gpx
1,002,Röttgen-Buschhoven-Apfelmaar,52,304,https://rdspt.com/inlawsgravel/gpxs/002.gpx
2,003,Messdorf-Apfelmaar-Kottenforst-Bad Godesberg,51,371,https://rdspt.com/inlawsgravel/gpxs/003.gpx
3,004,Drachenfelserländchen-Rheinufer V1,42,428,https://rdspt.com/inlawsgravel/gpxs/004.gpx
4,005,$InlawsGravel05_05_,34,356,https://rdspt.com/inlawsgravel/gpxs/005.gpx
5,006,$InlawsGravel13_05_,35,311,https://rdspt.com/inlawsgravel/gpxs/006.gpx
6,007,Kessenich-Bahnhof Kottenforst und züruck,39,209,https://rdspt.com/inlawsgravel/gpxs/007.gpx
7,008,Tour divide 1: Kottenforst-Bad Go Rheinfähre -...,37,373,https://rdspt.com/inlawsgravel/gpxs/008.gpx
8,009,Dollendorfer Hardt-Klöster Heisterbach-Lauterb...,33,371,https://rdspt.com/inlawsgravel/gpxs/009.gpx
9,010,Alfter-Apfelmaar-Röttgen-Kottenforst,34,291,https://rdspt.com/inlawsgravel/gpxs/010.gpx
